<a href="https://colab.research.google.com/github/SamuelBFG/DL-studies/blob/master/2_similarity_search_level_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before we start you need to add the Caltech 101 dataset to your Google Colab enviroment. Here is the [link](https://www.kaggle.com/ceciliala/caltech-101).

In [1]:
# Kaggle dependencies will already be installed so there is no need for "!pip install kaggle"
# You'll need to upload your kaggle.json file though

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"samuelbfg","key":"0720c91e1f13c46c81f9cf850f621e4e"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Change the permission
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d ceciliala/caltech-101

 99% 114M/115M [00:00<00:00, 82.5MB/s] 
100% 115M/115M [00:01<00:00, 119MB/s] 


In [4]:
from zipfile import ZipFile
file_name = "caltech-101.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


Here we are going to use the files in the ***data*** folder we obtained from the previous notebook ***1-feature-extraction***

The files that we are going to need are:

```
   class_ids-caltech101.pickle
   features-caltech101-resnet.pickle
   filenames-caltech101.pickle
```

In [5]:
from zipfile import ZipFile
file_name = "data.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


## **Level 2**

We benchmark the algorithms based on the time it takes to index images and locate the most similar image based on its features using the Caltech-101 dataset. We also experiment with t-SNE and PCA.

### **Understanding the time it takes to index images and locate the most similar image based on its features**

For these experiments we will use the features of the Caltech101 dataset that we read above.

First, let's choose a random image to experiment with. We will be using the same image for all the following experiments. Note: the results may change if the image is changed.

In [6]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [15]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-resnet.pickle',
                                'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

In [16]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  2048


In [17]:
random_image_index = random.randint(0, num_images)

**Standard features**

The following experiments are based on the ResNet-50 features derived from the images of the Caltech101 dataset.

### **Standard features + Brute Force Algorithm on one image**

We will be timing the indexing for various Nearest Neighbors algorithms, so let's start with timing the indexing for the Brute force algorithm. While running terminal commands in iPython like the timeit command, the variables are not stored in memory, so we need to rerun the same command to compute and store the results in the variable.

In [18]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)

100 loops, best of 3: 17.9 ms per loop


Now, let's look at the time it takes to search for the nearest neighbors for the selected random image using the trained model with the Brute force algorithm.

In [19]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

10 loops, best of 3: 93.3 ms per loop


### **Standard features + k-d Tree Algorithm on one image**

Now let's turn our attention to the next nearest neighbors algorithm, the k-d tree. Let's time the indexing for the k-d tree algorithm.

In [20]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)

1 loop, best of 3: 3.51 s per loop



Now, time the search for the same random image using the k-d tree trained model.

In [21]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

10 loops, best of 3: 45 ms per loop


We will increase the number of our test images so that we can experiment with how the scalability of different nearest neighbors algorithms change. Let's choose a random set of 100 or 1000 images to experiment.

Note: the results may change if any of the images are changed

Generate a list of images to do the next set of experiments on.

In [22]:
random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list = [
    feature_list[each_index] for each_index in random_image_indices
]

**Standard features + Brute Force Algorithm on a set of images**

Time the search for the Brute force algorithm.

In [23]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 3: 1.46 s per loop


**Standard features + k-d Tree Algorithm on a set of images**

Time the search for the k-d tree algorithm.

In [24]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 3: 43.9 s per loop


**Standard features + Ball Tree Algorithm on a set of images**

Time the search for the Ball Tree algorithm.

In [25]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 3: 32.4 s per loop


### **PCA**

Now we have seen the time it takes to index and search using nearest neighbor algorithms on the full feature length. We can use PCA to compress the features and reduce the time. As before we set the number of features intended.

In [26]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

Train the PCA model with the number of desired feature dimensions.

In [27]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

Let's try to understand the importance of each of the resultant features. The numbers displayed below show the relative importance of the first 20 features.

In [28]:
print(pca.explained_variance_ratio_[0:20])

[0.06110182 0.04382477 0.04060571 0.0322854  0.02124299 0.01967342
 0.01750924 0.01519273 0.01506694 0.01313027 0.01261716 0.01226299
 0.01129626 0.01055883 0.00959002 0.0093974  0.00869048 0.00849483
 0.00836702 0.00772747]


### **PCA + Brute Force Algorithm on one image**

Let's time the indexing for the brute force algorithm.

In [29]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)

10 loops, best of 3: 39 ms per loop



We will now time the search for the brute force algorithm.

In [30]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

100 loops, best of 3: 2.15 ms per loop


### **PCA + k-d Tree Algorithm on one image**

Time the indexing for the k-d tree algorithm.

In [31]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)

10 loops, best of 3: 131 ms per loop



Time the search for the k-d tree algorithm.

In [32]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

1000 loops, best of 3: 772 µs per loop


### **PCA + Ball Tree Algorithm on one image**

Time the indexing for the ball tree algorithm.

In [33]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)

10 loops, best of 3: 97.5 ms per loop



Time the search for the ball tree algorithm.

In [34]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

1000 loops, best of 3: 1.89 ms per loop


We use the same random indices to experiment. Note: the results may change if any of the images are changed.

Generate a list of images to do the next set of experiments on.

In [35]:
random_feature_list_compressed = [
    feature_list_compressed[each_index] for each_index in random_image_indices
]

### PCA + Brute Force Algorithm on a set of images
Time the search for the brute force algorithm.

In [36]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 3: 214 ms per loop


### **PCA + k-d Tree Algorithm on a set of images**

Time the search for the k-d tree algorithm.

In [37]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 3: 1.52 s per loop


### **PCA + Ball Tree Algorithm on a set of images**

Time the search for the Ball Tree algorithm.

In [38]:
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 3: 1.33 s per loop


### **Annoy**

Make sure you have annoy installed. You can install it using pip, pip3 install 
annoy.

In [39]:
!pip3 install annoy

     |████████████████████████████████| 655kB 2.7MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp36-cp36m-linux_x86_64.whl size=390344 sha256=657e8ad41e96a9fb5f5746bce011de06f056c7c1bc2d9cc97cf1eb7b7fa31a75
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [40]:
from annoy import AnnoyIndex

In [41]:
# Time the indexing for Annoy
t = AnnoyIndex(2048)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images):
    feature = feature_list[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


2.4121952056884766


True

**Annoy on one image**

Time the search for one image for Annoy.

In [42]:
u = AnnoyIndex(2048)
%timeit u.get_nns_by_vector(feature_list[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list[random_image_index],
                              5,
                              include_distances=True)

1000 loops, best of 3: 282 µs per loop


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.

In [43]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(feature_list[random_image_index],
                                      5,
                                      include_distances=True)

### **Annoy on a set of images**

Time the search for multiple images for Annoy.

In [44]:
%time calculate_annoy_time()

CPU times: user 31.4 ms, sys: 65 µs, total: 31.5 ms
Wall time: 30.9 ms


### **PCA + Annoy**

Now, let's time the indexing for Annoy for the PCA generated features.

In [45]:
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  This is separate from the ipykernel package so we can avoid doing imports until


0.026290416717529297


True

**PCA + Annoy for one image**

Time the search for one image for Annoy.

In [46]:
u = AnnoyIndex(num_feature_dimensions)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


The slowest run took 12.18 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.04 µs per loop


Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.

In [47]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(
            feature_list_compressed[random_image_index],
            5,
            include_distances=True)

**PCA + Annoy on a set of images**

Time the search for multiple images for Annoy.

In [48]:
%time calculate_annoy_time()

CPU times: user 369 µs, sys: 7 µs, total: 376 µs
Wall time: 382 µs


### **NMS Lib**

In [49]:
!pip install nmslib

     |████████████████████████████████| 13.0MB 325kB/s 
     |████████████████████████████████| 296kB 46.7MB/s 


In [50]:
import nmslib

In [51]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(feature_list_compressed)
index.createIndex({'post': 2}, print_progress=True)

NMS Lib on one image

In [52]:
# Query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(feature_list_compressed[random_image_index], k=5)
ids, distances = index.knnQuery(feature_list_compressed[random_image_index],
                                k=5)

The slowest run took 6.59 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 20.6 µs per loop


NMS Lib on a set of images

In [53]:
# Get all nearest neighbors for all the datapoint
# using a pool of 4 threads to compute
%timeit index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)
neighbors = index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)

10 loops, best of 3: 180 ms per loop


### **Falconn**

In [54]:
!pip install falconn

     |████████████████████████████████| 1.4MB 2.8MB/s 
  Created wheel for falconn: filename=FALCONN-1.3.1-cp36-cp36m-linux_x86_64.whl size=10581705 sha256=8a12dddb813a6decc2924aa9bd2304972810d4fda2efbc981cdfd8f5500e3434
  Stored in directory: /root/.cache/pip/wheels/bf/36/96/d5538901888620fc0343c1ed9d5f87fce00869e00c12056ef8
Successfully built falconn


In [55]:
import falconn

In [56]:
# Setup different parameters for Falonn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_feature_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable

# Train the Falconn model
falconn.compute_number_of_hash_functions(16, parameters)

**Falconn on a set of images**

In [57]:
dataset = np.array(feature_list_compressed)
a = np.random.randn(8677, 100)
a /= np.linalg.norm(a, axis=1).reshape(-1, 1)
dataset = a

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

searchQuery = np.array(feature_list_compressed[random_image_index])
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

CPU times: user 11.7 ms, sys: 956 µs, total: 12.6 ms
Wall time: 14.8 ms
The slowest run took 199.80 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.16 µs per loop


**PCA + Annoy**

In [58]:
# Time the indexing for Annoy for the PCA generated features
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = dataset[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

0.11941123008728027


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  after removing the cwd from sys.path.


True

In [59]:
u = AnnoyIndex(num_feature_dimensions)
# Time the search for one image for Annoy
%timeit u.get_nns_by_vector(dataset[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(dataset[random_image_index],
                              5,
                              include_distances=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


10000 loops, best of 3: 12.4 µs per loop
